# Загрузка текста

In [ ]:
import tensorflow as tf

import tensorflow_datasets as tfds
import os

Тексты трех переводов выполнили:

 - [Вильям Купер](https://en.wikipedia.org/wiki/William_Cowper) — [текст](https://storage.googleapis.com/download.tensorflow.org/data/illiad/cowper.txt)

 - [Эрл Эдвард](https://en.wikipedia.org/wiki/Edward_Smith-Stanley,_14th_Earl_of_Derby) — [текст](https://storage.googleapis.com/download.tensorflow.org/data/illiad/derby.txt)

- [Сэмюель Батлер](https://en.wikipedia.org/wiki/Samuel_Butler_%28novelist%29) — [текст](https://storage.googleapis.com/download.tensorflow.org/data/illiad/butler.txt)

Текстовые файлы использованные в этом руководстве были подвергнуты некоторым типичным задачам предварительной обработки, в основном удаление материала - шапка и футер документа, нумерация строк, заголовки глав. Скачайте эти файлы локально.

In [ ]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

811008/807992 [==============================] - 0s 0us/step


'/root/.keras/datasets'

## Загрузите текст в датасеты

Переберите файлы, загружая каждый в свой датасет.

Каждый пример нужно пометить индивидуально, так что используйте `tf.data.Dataset.map` чтобы применить функцию расставляющую метки каждому элементу. Она переберет каждую запись в датасете возвращая пару (`example, label`).

In [ ]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

Объедините эти размеченные наборы данных в один и перемешайте его.


In [ ]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [ ]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

Вы можете использовать `tf.data.Dataset.take` и `print`, чтобы посмотреть как выглядят пары `(example, label)`. Свойство `numpy` показывает каждое значение тензора.

In [ ]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'More than a woman or a puny child:'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'The Trojans, Hector and \xc3\x86neas rush'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'"Hector, thou son of Priam, leave me not'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Some say hereafter, Menelaus bore'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"To avenge her slaughter'd brothers on his head.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


## Закодируйте текстовые строки числами

Модели машинного обучения работают с числами, не словами, так что строковые значения необходимо конвертировать в списки с числами. Чтобы сделать это поставьте в соответствие каждому слову свое число.

### Создайте словарь

Сперва создайте словарь токенизировав текст в коллекцию отдельных отличающихся слов. Есть несколько способов сделать это и в TensorFlow и в Python. В этом учебнике:

1. Переберите `numpy` значения всех примеров.
2. Используйте `tfds.features.text.Tokenizer` чтобы разбить их на токены.
3. Соберите эти токены в множество Python чтобы избавиться от дубликатов
4. Получите размер словаря для последующего использования.

In [ ]:
tokenizer = tfds.deprecated.text.Tokenizer()
# tf.keras.preprocessing.text.Tokenizer()
tf.keras.preprocessing.sequence.
vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

17178

In [ ]:
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set)

Вы можете посмотреть одну строку чтобы увидеть как выглядит результат работы кодировщика.

In [ ]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b'More than a woman or a puny child:'


In [ ]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[4385, 13505, 5499, 3662, 3117, 5499, 5549, 14418]


Теперь запустите кодировщик на датасете обернув его в `tf.py_function` и передав в метод `map` датасета.

In [ ]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

## Разбейте датасет на тестовую и обучающую выборки

Используйте `tf.data.Dataset.take` и `tf.data.Dataset.skip` чтобы создать небольшой тестовый и большой обучающий датасеты.

Перед передачей в модель датасет должны быть разбиты на пакеты. Обычно количество записей в пакете и их размерность должно быть одинаковые.

In [ ]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

Сейчас, `test_data` и `train_data` являются не коллекциями пар (`example, label`), а коллекциями пакетов. Каждый пакет это пара вида (*много примеров*, *много меток*) представленная в виде массивов.

Чтобы проиллюстрировать:

In [ ]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

(<tf.Tensor: shape=(17,), dtype=int64, numpy=
 array([ 4385, 13505,  5499,  3662,  3117,  5499,  5549, 14418,     0,
            0,     0,     0,     0,     0,     0,     0,     0])>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>)

Так как мы ввели новую кодировку токенов (нуль использовался для заполнения), размер словаря увеличился на единицу.

In [ ]:
vocab_size += 1

## Постройте модель


In [ ]:
model = tf.keras.Sequential()

In [ ]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

In [ ]:
model.add(tf.keras.layers.GlobalAveragePooling1D())

In [ ]:
# Один или более плотных слоев.
# Отредактируйте список в строке `for` чтобы поэкспериментировать с размером слоев.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Выходной слой. Первый аргумент - число меток.
model.add(tf.keras.layers.Dense(3, activation='softmax'))

Наконец скомпилируйте модель. Для softmax категоризационной модели используйте `sparse_categorical_crossentropy` в виде функции потерь. Вы можете попробовать другие оптимизаторы, но `adam` очень часто используемая.

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          1099456   
_________________________________________________________________
global_average_pooling1d (Gl (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 1,107,971
Trainable params: 1,107,971
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Обучите модель


In [ ]:
model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3
697/697 [==============================] - 11s 16ms/step - loss: 0.5495 - accuracy: 0.7392 - val_loss: 0.4135 - val_accuracy: 0.8146
Epoch 2/3
697/697 [==============================] - 11s 16ms/step - loss: 0.3182 - accuracy: 0.8615 - val_loss: 0.3790 - val_accuracy: 0.8302
Epoch 3/3
697/697 [==============================] - 12s 17ms/step - loss: 0.2423 - accuracy: 0.8952 - val_loss: 0.3919 - val_accuracy: 0.8330


In [ ]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

79/79 [==============================] - 2s 23ms/step - loss: 0.3701 - accuracy: 0.8384

Eval loss: 0.370, Eval accuracy: 0.838
